<a href="https://colab.research.google.com/github/anon560096/bds_project/blob/master/code/data_preprocessing_for_mongodb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The cells below are for finding out the unique cities in business.json

In [0]:
import json

In [0]:
cities = []
with open('business.json', 'r') as f:
  for line in f:
    business = json.loads(line)
    cities.append(business['city'])
f.close()

In [0]:
city_count = {}
for city in cities:
  if city in city_count:
    city_count[city] += 1
  else:
    city_count[city] = 1

In [0]:
with open('city_count.txt', 'w+') as f:
  for city, count in city_count.items():
    new_line = "City: " + str(city) + " Count: " + str(count)
    print(new_line)
    f.write(new_line)
f.close()

In [0]:
import operator
sorted_cities = sorted(city_count.items(), key=operator.itemgetter(1), reverse=True)
sorted_cities = [city[0] for city in sorted_cities[:30]]
print(sorted_cities)

The cells below are for preprocessing of JSON files

In [0]:
import pymongo
from pprint import pprint
import json
from pymongo import MongoClient, GEO2D
import requests
import json

In [0]:
def setup_python():
  business = {}
  labels = {}
  photos = {}
  num_photo = 0
  num_matched = 0
  yes_hours = 0
  no_hours = 0

  with open('business.json') as f:
    for line in f:
      b = json.loads(line)
      business[b['business_id']] = b

  with open('photo.json') as f:
    for line in f:
      p = json.loads(line)
      photos[p['photo_id']] = p
  
  with open('labels.json') as f:
    for line in f:
      l = json.loads(line)
      labels[l['id']] = l

  with open('photos_new.json', 'w+') as f: 
    for photo_id, photo in photos.items():
      num_photo = num_photo + 1
      data = business[photo['business_id']]
      label = labels[photo_id]
      if data != None:
        num_matched = num_matched + 1
        stars = data['stars']
        longitude = data['longitude']
        latitude = data['latitude']
        hours = None
        business_id = data['business_id']
        city = data['city']
        categories_temp = data['categories'].split(',')
        categories = []
        for c in categories_temp:
          categories.append(c.strip())

        if 'hours' not in data:
          no_hours = no_hours + 1
        else:
          hours = data['hours']

        photo_new = photo

        if hours != None :
          temp = hours.get(list(hours.keys())[0]).split("-")
          opening_time = temp[0].split(':')
          open_hours = int(opening_time[0])
          open_hours = (open_hours * 60) + int(opening_time[1])

          closing_time = temp[1].split(':')
          close_hours = int(closing_time[0])
          close_hours = (close_hours*60) + int(closing_time[1])

          # db.photos.update_many({'business_id': photo['business_id']},{'$set': {'opening_time': int(open_hours),'closing_time': int(close_hours)}},upsert=False)
          photo_new['opening_time'] = int(open_hours)
          photo_new['closing_time'] = int(close_hours)

          yes_hours = yes_hours + 1

        coord = [float(longitude), float(latitude)]
        photo_new['business rating'] = stars
        photo_new['categories'] = categories
        photo_new['loc'] = {'coordinates':coord, 'type':"Point"}
        photo_new['101_label'] = label['101_label']
        photo_new['city'] = city

        f.write(json.dumps(photo_new) + '\n')
        # db.photos.update_many({'business_id': photo['business_id']},{'$set': {'business rating': stars, 'business id': stars, 'categories': categories,
        #           'loc': {'coordinates':coord, 'type':"Point"}}}, upsert=False)
    
        # db.photos.update_many({'photo_id': photo['photo_id']}, {'$set': {'101_label': label['101_label']}})



In [0]:
import json
setup_python()